<a href="https://colab.research.google.com/github/ohki-yu0225/social_media_analysis/blob/main/network_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ソーシャルメディア分析・入門(2)：ネットワーク分析演習

【内容】
- ネットワークの可視化
- 次数分布
- 中心性解析
- コミュニティ解析
- クラスター係数/最短経路長

---
## ライブラリのインポート

Pythonではネットワーク分析のためのライブラリである`networkx`が利用できる。`networkx`は`nx`という略称でインポートする。

In [ ]:
!pip install japanize_matplotlib
!pip install igraph
!pip install leidenalg

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib
import networkx as nx
import igraph as ig
import leidenalg as la

color_map = plt.cm.get_cmap('Set1')

---
## ネットワークの可視化

`networkx`では，ネットワークを`Graph`オブジェクトとして扱う。`nx.draw_networkx`関数でネットワークを可視化する。

In [ ]:
G = nx.karate_club_graph() # 空手クラブネットワークを読み込み
pos = nx.spring_layout(G, seed=42) # レイアウトを計算

# ネットワークを描画
plt.figure(figsize=(4,3))
nx.draw_networkx(G, pos)
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

ネットワーク上の点をノード，ノード間の関係性を表す線をエッジという。グラフオブジェクトのノード・エッジ情報は`nodes`メソッドや`edges`メソッドで取得できる。

In [ ]:
G.nodes()

In [ ]:
G.edges()

In [ ]:
print(f"ノード数：{len(G.nodes())}")
print(f"エッジ数：{len(G.edges())}")

---
## 次数分布

各ノードのエッジの数を次数と呼ぶ。各ノードの次数は`degree`メソッドで取得する。

In [ ]:
G.degree()

次数の分布（次数分布）はネットワークの全体的な構造を表しており，特に平均次数はネットワークの平均的なつながりの数を表す特徴量である。平均次数は次の式で計算できる。

$$
\text{（平均次数）} = \frac{2 \times \text{（エッジ数）}}{\text{（ノード数）}}
$$

In [ ]:
degrees = [d for n, d in G.degree()]
print(f"平均次数：{2*len(G.edges()) / len(G.nodes())}")

plt.figure(figsize=(4, 3))
plt.hist(degrees, bins=np.arange(min(degrees), max(degrees) + 2) - 0.5,
         edgecolor="black", linewidth=1)
plt.xlabel('次数')
plt.ylabel('ノード数')
plt.xticks(np.arange(min(degrees), max(degrees) + 1))
plt.show()

---
## 中心性解析

中心性とはノードの重要さを定量的に表すものであり、様々な種類の中心性が考案されている。ネットワークのノードの中心性を計算し、各ノードの特徴を調べることを中心性解析という。

### 次数中心性

先述の次数が高いノードはネットワーク上で多くのノードとつながっているハブのような存在であることを示す。ハブノードが重要であるという考え方に基づくと，次数は中心性として用いることができる（次数中心性）。

In [ ]:
degrees = [d for n, d in G.degree()]

plt.figure(figsize=(4,3))
nx.draw_networkx(G, pos, node_size=[i * 10 for i in degrees])
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

### 媒介中心性

媒介中心性はネットワーク上の流れを橋渡しする度合いを計測する中心性である。ネットワーク上の2つのノードを結ぶ最短経路を全ての組み合わせについて計算し、あるノードが最短経路上のパスに含まれている割合として定義される。媒介中心性は`nx.betweenness_centrality()`で計算できる。

In [ ]:
betweenness_centralities = list(nx.betweenness_centrality(G).values())

plt.figure(figsize=(4,3))
nx.draw_networkx(G, pos, node_size=[i * 300 for i in betweenness_centralities])
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

次数中心性や媒介中心性が高いノードはネットワーク上で重要な役割を果たすことが多いが，それぞれの中心性は重要度の定義が異なる。そのため，複数の中心性を比較することで，それぞれのノードが持つ特徴を調べることができる。

In [ ]:
plt.figure(figsize=(4, 3))
plt.scatter(degrees, betweenness_centralities)
plt.xlabel("次数")
plt.ylabel("媒介中心性")
plt.show()

---
## コミュニティ解析

ネットワーク上で同じグループのノードの中で密にエッジがあり，異なるグループ間ではあまりエッジがない構造が存在するとき，それぞれのグループをコミュニティと呼ぶ。コミュニティ構造を検出するための手法がコミュニティ解析である。代表的なコミュニティ解析手法であるLouvain法は`nx.community.louvain_communities`関数で実行できる。

In [ ]:
communities = nx.community.louvain_communities(G)
print(f"コミュニティ数：{len(communities)}")

node_communities = {}

for idx, community_nodes in enumerate(communities):
    for node in community_nodes:
        node_communities[node] = idx

node_colors = [color_map(node_communities[node]) for node in G.nodes()]
edge_colors = []
for u, v in G.edges():
    if node_communities[u] == node_communities[v]:
        edge_colors.append(color_map(node_communities[u]))
    else:
        edge_colors.append("lightgray")

plt.figure(figsize=(4,3))
nx.draw_networkx(G, pos, node_color=node_colors, edge_color=edge_colors)
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()


Louvain法では，ネットワークをコミュニティに分割する際の解像度(resolution)を調整することができる。`resolution`パラメータで解像度を指定する。

In [ ]:
communities = nx.community.louvain_communities(G, resolution=1)
print(f"コミュニティ数：{len(communities)}")

node_communities = {}

for idx, community_nodes in enumerate(communities):
    for node in community_nodes:
        node_communities[node] = idx

node_colors = [color_map(node_communities[node]) for node in G.nodes()]
edge_colors = []
for u, v in G.edges():
    if node_communities[u] == node_communities[v]:
        edge_colors.append(color_map(node_communities[u]))
    else:
        edge_colors.append("lightgray")

plt.figure(figsize=(4,3))
nx.draw_networkx(G, pos, node_color=node_colors, edge_color=edge_colors)
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()


中心性解析とコミュニティ解析を組み合わせて，各コミュニティ内で次数中心性が大きなノードを調べる。

In [ ]:
plt.figure(figsize=(4,3))
nx.draw_networkx(G, pos, node_color=node_colors, edge_color=edge_colors, node_size=[i * 10 for i in degrees])
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
# 次数とコミュニティの情報をまとめる
df = pd.DataFrame({"degree": degrees,
                   "community": [node_communities[n] for n in G.nodes]}, index=G.nodes())
df

In [ ]:
# コミュニティ0のノードの次数を調べる
df[df["community"]==0].sort_values(by="degree", ascending=False).head(5)

In [ ]:
# コミュニティ1のノードの次数を調べる
df[df["community"]==1].sort_values(by="degree", ascending=False).head(5)

【参考】ザッカリーの空手クラブ([Wikipedia](https://en.wikipedia.org/wiki/Zachary%27s_karate_club))

【発展】Louvain法を改良した手法にLeiden法があり，Leiden法も広く使われている。

In [ ]:
# Leiden法を実行するための関数を定義
def leiden_communities(G, resolution=1.0):
    nodes = list(G.nodes())
    idx = {u: i for i, u in enumerate(nodes)}
    edges = [(idx[u], idx[v]) for u, v in G.edges()]

    g = ig.Graph(n=len(nodes), edges=edges, directed=False)

    part = la.find_partition(
        g,
        la.RBConfigurationVertexPartition,
        resolution_parameter=resolution,
    )

    return [set(nodes[i] for i in comm) for comm in part]

In [ ]:
communities = leiden_communities(G, resolution=1)
print(f"コミュニティ数：{len(communities)}")
node_communities = {}

for idx, community_nodes in enumerate(communities):
    for node in community_nodes:
        node_communities[node] = idx

node_colors = [color_map(node_communities[node]) for node in G.nodes]
edge_colors = []
for u, v in G.edges():
    if node_communities[u] == node_communities[v]:
        edge_colors.append(color_map(node_communities[u]))
    else:
        edge_colors.append("lightgray")

plt.figure(figsize=(4,3))
nx.draw_networkx(G, pos, node_color=node_colors, edge_color=edge_colors)
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

---
## 演習：アメリカ合衆国議員議員のTwitterネットワーク分析

データの概要：第117期アメリカ合衆国議員のTwitter上での相互作用（リツイート，引用リツイート，返信，メンション）を表すネットワーク。

出典：C.G. Fink, N. Omodt, S. Zinnecker, and G. Sprint: A Congressional Twitter network dataset quantifying pairwise probability of influence. Data in Brief, 2023.（[https://snap.stanford.edu/data/congress-twitter.html](https://snap.stanford.edu/data/congress-twitter.html)からデータを取得し，本演習用に各議員の名前，政党（共和党/民主党，独立系の場合は会派に分類），議会（上院/下院）をノード情報として加えたgraphml形式とした。所属政党はOpenAI APIのgpt-4.1-miniモデルを用いて，Twitterハンドルから推定し，不明と推定されたものはウェブ上の情報から特定した。）

In [ ]:
G = nx.read_graphml("congress_network.graphml")
G = nx.relabel_nodes(G, int)

In [ ]:
df = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient="index").sort_index()
df

演習1：ノード数とエッジ数を調べる。

In [ ]:
degrees = [d for n, d in G.degree()]
print(f"平均次数：{2*len(G.edges()) / len(G.nodes())}")

plt.figure(figsize=(4, 3))
plt.hist(degrees, bins=np.arange(min(degrees), max(degrees) + 2, 5) - 0.5,
         edgecolor="black", linewidth=1)
plt.xlabel('次数')
plt.ylabel('ノード数')
#plt.xticks(np.arange(min(degrees), max(degrees) + 1))
plt.show()

---
### ネットワークの可視化

In [ ]:
pos = nx.spring_layout(G, seed=42)

plt.figure(figsize=(5, 5))
nx.draw_networkx_nodes(G, pos, node_size=5)
nx.draw_networkx_edges(G, pos, width=0.2, alpha=0.3, node_size=50)
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
# 民主党員は青，共和党員は赤で示す
def set_colormap_party(G):
    party_attr = nx.get_node_attributes(G, 'party')
    color_map = []
    for node in G.nodes():
        party = party_attr.get(node)
        if party == 'Democrat':
            color_map.append('blue')
        else:
            color_map.append('red')
    return color_map

plt.figure(figsize=(5, 5))
nx.draw_networkx_nodes(G, pos, node_size=5, node_color=set_colormap_party(G))
nx.draw_networkx_edges(G, pos, width=0.2, alpha=0.3, node_size=50)
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
# 上院は緑，下院はオレンジで示す
def set_colormap_chamber(G):
    party_attr = nx.get_node_attributes(G, 'chamber')
    color_map = []
    for node in G.nodes():
        party = party_attr.get(node)
        if party == 'House':
            color_map.append('green')
        else:
            color_map.append('orange')
    return color_map

plt.figure(figsize=(5, 5))
nx.draw_networkx_nodes(G, pos, node_size=5, node_color=set_colormap_chamber(G))
nx.draw_networkx_edges(G, pos, width=0.2, alpha=0.3, node_size=50)
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

---
## 中心性解析

In [ ]:
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
degree_centralities = np.array([v for k, v in G.degree()])
nx.draw_networkx_nodes(G, pos, node_size = degree_centralities / max(degree_centralities) * 30, node_color=set_colormap_party(G))
nx.draw_networkx_edges(G, pos, width=0.2, alpha=0.3, node_size=50)
plt.axis('off')
plt.tight_layout(pad=0)
plt.title("次数中心性")

plt.subplot(1, 2, 2)
betweenness_centralities = np.array(list(nx.betweenness_centrality(G).values()))
nx.draw_networkx_nodes(G, pos, node_size = betweenness_centralities / max(betweenness_centralities) * 30, node_color=set_colormap_party(G))
nx.draw_networkx_edges(G, pos, width=0.2, alpha=0.3, node_size=50)
plt.axis('off')
plt.tight_layout(pad=0)
plt.title("媒介中心性")

In [ ]:
df["degree"] = dict(G.degree())
df["betweenness_centrality"] = nx.betweenness_centrality(G)
df

演習2：次数中心性が高い上位10人の議員を調べる。

演習3：媒介中心性が高い上位10人の議員を調べる。

【参考】第117期アメリカ合衆国議員([Wikipedia](https://en.wikipedia.org/wiki/117th_United_States_Congress#Leadership))

---
## コミュニティ解析

In [ ]:
communities = leiden_communities(G, resolution=1)
print(f"コミュニティ数：{len(communities)}")
node_communities = {}

for idx, community_nodes in enumerate(communities):
    for node in community_nodes:
        node_communities[node] = idx

node_colors = [color_map(node_communities[node]) for node in G.nodes]
edge_colors = []
for u, v in G.edges():
    if node_communities[u] == node_communities[v]:
        edge_colors.append(color_map(node_communities[u]))
    else:
        edge_colors.append("lightgray")

plt.figure()
nx.draw_networkx_nodes(G, pos, node_size=5, node_color=node_colors)
nx.draw_networkx_edges(G, pos, width=0.2, alpha=0.3, node_size=50, edge_color=edge_colors)
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
df["community"] = node_communities
df

In [ ]:
pd.crosstab(df["community"], df["party"])

In [ ]:
pd.crosstab(df["community"], df["chamber"])

演習4：ノードの色が政党，エッジの色がコミュニティを表すようにネットワークを描画する。

In [ ]:
plt.figure()
# ---コードを記述---


# ---コードを記述---
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

演習5：ノードの色が議会，エッジの色がコミュニティを表すようにネットワークを描画する。

In [ ]:
plt.figure()
# ---コードを記述---


# ---コードを記述---
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

【発展】政党をノードの色，議会をノードの形で表現する。

In [ ]:
G_house = G.subgraph(df[df["chamber"]=="House"].index)
G_senate = G.subgraph(df[df["chamber"]=="Senate"].index)

plt.figure()
nx.draw_networkx_nodes(G_house, pos, node_size=5, node_color=set_colormap_party(G_house), node_shape="^") # 下院は三角
nx.draw_networkx_nodes(G_senate, pos, node_size=5, node_color=set_colormap_party(G_senate), node_shape="s") # 上院は四角
nx.draw_networkx_edges(G, pos, width=0.2, alpha=0.3, node_size=50, edge_color=edge_colors)
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()


演習6：各コミュニティで次数中心性が高い上位5人の議員を調べる。